In [ ]:
import json 
from collections import defaultdict

def compare_labels_statistically(real_metadata, gemini_labeled_metadata):
    """
    Compares real (ground truth) sentence labels with Gemini's hypothetical labels
    and calculates precision, recall, and F1-score for each label type,
    with special handling for Sensitive/Confidential misclassifications.

    Args:
        real_metadata (list of dict): The ground truth metadata with 'paragraph_id',
                                      'sentences' (list of dict with 'sentence', 'label').
        gemini_labeled_metadata (list of dict): The metadata representing Gemini's labels,
                                                in the same structure as real_metadata.

    Returns:
        dict: A dictionary containing overall accuracy and per-label precision, recall, and F1-score.
              Returns None if inputs are invalid or empty.
    """
    if not real_metadata or not gemini_labeled_metadata:
        print("Error: Input metadata cannot be empty.")
        return None

    # Flatten the metadata into a list of (sentence_text, real_label, gemini_label) tuples
    # We assume sentence order and content are identical for comparison
    flat_real_labels = {}
    for para in real_metadata:
        paragraph_id = para.get("paragraph_id")
        for i, sent_data in enumerate(para.get("sentences", [])):
            sentence_text = sent_data.get("sentence")
            label = sent_data.get("label")
            if sentence_text and label:
                # Use a unique key for each sentence (paragraph_id, sentence_index)
                flat_real_labels[(paragraph_id, i)] = {'sentence': sentence_text, 'label': label}

    flat_gemini_labels = {}
    for para in gemini_labeled_metadata:
        paragraph_id = para.get("paragraph_id")
        for i, sent_data in enumerate(para.get("sentences", [])):
            sentence_text = sent_data.get("sentence")
            label = sent_data.get("label")
            if sentence_text and label:
                flat_gemini_labels[(paragraph_id, i)] = {'sentence': sentence_text, 'label': label}

    # Ensure both lists have the same number of sentences for a fair comparison
    if len(flat_real_labels) != len(flat_gemini_labels):
        print("Warning: Number of sentences in real metadata and Gemini metadata differ.")
        print(f"Real sentences: {len(flat_real_labels)}, Gemini sentences: {len(flat_gemini_labels)}")
        # For a robust comparison, we should only compare sentences that exist in both.
        # For simplicity here, we'll only compare sentences present in both.
        # In a real scenario, you might want to identify missing/extra sentences.

    all_labels = sorted(list(set([s['label'] for s in flat_real_labels.values()])))
    # Initialize counts for True Positives, False Positives, False Negatives for each label
    metrics = {label: {'TP': 0, 'FP': 0, 'FN': 0} for label in all_labels}
    total_correct = 0
    total_sentences = 0

    # Iterate through the real labels and compare with Gemini's labels
    for key, real_sent_data in flat_real_labels.items():
        if key not in flat_gemini_labels:
            # This sentence from real_metadata is missing in gemini_labeled_metadata
            # For simplicity, we'll skip it for metric calculation, but note it.
            continue

        real_label = real_sent_data['label']
        gemini_label = flat_gemini_labels[key]['label']
        total_sentences += 1

        if real_label == gemini_label:
            # Standard correct classification
            metrics[real_label]['TP'] += 1
            total_correct += 1
        elif real_label == 'Sensitive' and gemini_label == 'Confidential':
            # Special rule: Sensitive predicted as Confidential is OK
            # Count as a True Positive for 'Sensitive' and contribute to overall correctness
            metrics['Sensitive']['TP'] += 1
            total_correct += 1
        else:
            # All other misclassifications are treated as errors
            # The real label is a False Negative
            metrics[real_label]['FN'] += 1
            # The predicted label is a False Positive
            metrics[gemini_label]['FP'] += 1

    results = {"overall_accuracy": 0}
    if total_sentences > 0:
        results["overall_accuracy"] = total_correct / total_sentences

    for label in all_labels:
        tp = metrics[label]['TP']
        fp = metrics[label]['FP']
        fn = metrics[label]['FN']

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        results[label] = {
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score,
            "TP": tp,
            "FP": fp,
            "FN": fn
        }
    return results

# --- Example Usage ---
# 1. Your 'real_metadata' from the previous immersive
real_metadata_with_confidential_json = """
[
  {
    "paragraph_id": 1,
    "sentences": [
      {"sentence": "Real estate investment involves the purchase, ownership, management, rental, and/or sale of real estate for profit.", "label": "Public"},
      {"sentence": "It is a popular asset class for wealth creation and diversification, offering potential for both income generation and capital appreciation.", "label": "Public"},
      {"sentence": "A client's specific investment portfolio details are highly confidential.", "label": "Confidential"},
      {"sentence": "Unlike traditional stock market investments, real estate can provide tangible assets that can be leveraged or improved.", "label": "Public"},
      {"sentence": "The market is influenced by various factors, including economic growth, interest rates, population changes, and local supply and demand dynamics.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 2,
    "sentences": [
      {"sentence": "Real estate investments can take many forms, each with its own risk and return profile.", "label": "Public"},
      {"sentence": "Common types include residential properties (single-family homes, multi-family units), commercial properties (office buildings, retail spaces), industrial properties (warehouses, factories), and raw land.", "label": "Public"},
      {"sentence": "Specialized categories like REITs (Real Estate Investment Trusts) offer a way to invest in real estate without direct property ownership.", "label": "Public"},
      {"sentence": "Understanding these different types is crucial for aligning investments with personal financial goals.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 3,
    "sentences": [
      {"sentence": "Investing in real estate offers several distinct advantages.", "label": "Public"},
      {"sentence": "These include potential for passive income through rentals, capital appreciation over time, and portfolio diversification.", "label": "Public"},
      {"sentence": "Real estate can also act as a hedge against inflation, as property values and rents tend to rise with inflation.", "label": "Public"},
      {"sentence": "Furthermore, investors can benefit from tax advantages, such as depreciation deductions and deductions for mortgage interest and property taxes.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 4,
    "sentences": [
      {"sentence": "While attractive, real estate investment is not without risks.", "label": "Public"},
      {"sentence": "These can include market downturns, liquidity issues (difficulty selling quickly), property damage, and tenant vacancies.", "label": "Sensitive"},
      {"sentence": "Interest rate fluctuations can impact mortgage costs and property valuations.", "label": "Sensitive"},
      {"sentence": "Local economic conditions, zoning changes, and environmental factors also pose risks.", "label": "Public"},
      {"sentence": "Through due diligence and risk assessment are essential before committing capital.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 5,
    "sentences": [
      {"sentence": "Most real estate investments are financed through a combination of equity (down payment) and debt (mortgages).", "label": "Public"},
      {"sentence": "Various mortgage options are available, including conventional loans, FHA loans, VA loans, and commercial loans.", "label": "Sensitive"},
      {"sentence": "Understanding loan-to-value ratios, interest rates, and amortization schedules is critical.", "label": "Sensitive"},
      {"sentence": "Some investors also explore hard money loans or private financing for specific situations, though these often come with higher interest rates.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 6,
    "sentences": [
      {"sentence": "Accurate property valuation is fundamental to making informed investment decisions.", "label": "Public"},
      {"sentence": "Common valuation methods include the sales comparison approach (comparing to similar recently sold properties), the income capitalization approach (based on potential rental income), and the cost approach (estimating replacement cost).", "label": "Public"},
      {"sentence": "Professional appraisals are often required for financing, but investors should also conduct their own market analysis.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 7,
    "sentences": [
      {"sentence": "Due diligence is a critical phase where potential investments are thoroughly investigated.", "label": "Public"},
      {"sentence": "This includes reviewing financial statements, property inspections, title searches, zoning compliance checks, and environmental assessments.", "label": "Sensitive"},
      {"sentence": "It's also important to understand local market trends, rental demand, and potential operating expenses.", "label": "Public"},
      {"sentence": "Neglecting due diligence can lead to unforeseen costs and poor investment outcomes.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 8,
    "sentences": [
      {"sentence": "Effective property management is key to maximizing returns from rental properties.", "label": "Public"},
      {"sentence": "This involves tenant screening, lease agreements, rent collection, property maintenance, and handling tenant issues.", "label": "Sensitive"},
      {"sentence": "Investors can choose to self-manage or hire a professional property management company.", "label": "Public"},
      {"sentence": "The choice depends on the investor's time availability, expertise, and the scale of their portfolio.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 9,
    "sentences": [
      {"sentence": "Real estate investments have significant tax implications that can impact profitability.", "label": "Public"},
      {"sentence": "Investors can deduct expenses such as mortgage interest, property taxes, insurance, and operating costs.", "label": "Sensitive"},
      {"sentence": "Depreciation is a non-cash deduction that can reduce taxable income.", "label": "Sensitive"},
      {"sentence": "Capital gains taxes apply upon sale, but 1031 exchanges can defer these taxes by reinvesting proceeds into similar properties.", "label": "Sensitive"},
      {"sentence": "Consulting a tax professional is highly recommended.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 10,
    "sentences": [
      {"sentence": "Real estate markets are cyclical, moving through phases of recovery, expansion, hyper supply, and recession.", "label": "Public"},
      {"sentence": "Understanding these cycles can help investors identify optimal times to buy, hold, or sell.", "label": "Public"},
      {"sentence": "Factors like interest rates, economic growth, and employment rates influence market phases.", "label": "Public"},
      {"sentence": "While timing the market perfectly is challenging, recognizing general trends can enhance investment strategies.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 11,
    "sentences": [
      {"sentence": "Leverage, or using borrowed money to increase potential returns, is a common strategy in real estate.", "label": "Public"},
      {"sentence": "While it can amplify profits, it also magnifies losses if the investment performs poorly.", "label": "Sensitive"},
      {"sentence": "Understanding the debt-to-equity ratio and managing mortgage payments are crucial.", "label": "Sensitive"},
      {"sentence": "Over-leveraging can lead to financial distress, especially during market downturns or unexpected vacancies.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 12,
    "sentences": [
      {"sentence": "REITs are companies that own, operate, or finance income-producing real estate.", "label": "Public"},
      {"sentence": "They offer a way to invest in large-scale real estate portfolios without direct property ownership, similar to mutual funds.", "label": "Public"},
      {"sentence": "REITs typically pay out a significant portion of their taxable income to shareholders as dividends, providing a steady income stream.", "label": "Sensitive"},
      {"sentence": "They offer liquidity and diversification benefits compared to direct property investment.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 13,
    "sentences": [
      {"sentence": "Real estate crowdfunding platforms allow multiple investors to pool their money to invest in larger real estate projects, such as commercial developments or multi-family properties.", "label": "Public"},
      {"sentence": "This lowers the barrier to entry for individual investors.", "label": "Public"},
      {"sentence": "While offering diversification and access to institutional-grade projects, investors should carefully vet platforms and understand the specific risks associated with each project.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 14,
    "sentences": [
      {"sentence": "Key metrics for evaluating real estate investments include cash flow (rental income minus expenses) and Return on Investment (ROI).", "label": "Public"},
      {"sentence": "Positive cash flow is essential for sustainable income.", "label": "Public"},
      {"sentence": "ROI calculations can vary (e.g., cash-on-cash return, cap rate) but generally measure the profitability of an investment relative to its cost.", "label": "Sensitive"},
      {"sentence": "A detailed financial analysis is crucial for projecting potential returns.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 15,
    "sentences": [
      {"sentence": "Having a clear exit strategy is as important as the initial investment plan.", "label": "Public"},
      {"sentence": "Common exit strategies include selling the property for a profit, refinancing to pull out equity, or holding the property for long-term rental income.", "label": "Sensitive"},
      {"sentence": "The chosen strategy depends on market conditions, the investor's financial goals, and tax considerations.", "label": "Sensitive"},
      {"sentence": "Planning ahead helps maximize returns and minimize unexpected challenges.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 16,
    "sentences": [
      {"sentence": "Real estate transactions involve numerous legal considerations.", "label": "Public"},
      {"sentence": "These include purchase agreements, property deeds, landlord-tenant laws, zoning regulations, and environmental laws.", "label": "Sensitive"},
      {"sentence": "Understanding local, state, and federal regulations is vital to avoid legal disputes and ensure compliance.", "label": "Public"},
      {"sentence": "Consulting with real estate attorneys is advisable for complex transactions or legal issues.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 17,
    "sentences": [
      {"sentence": "Macroeconomic indicators significantly influence the real estate market.", "label": "Public"},
      {"sentence": "Factors such as GDP growth, employment rates, inflation, and interest rates directly affect property values, rental demand, and investor confidence.", "label": "Public"},
      {"sentence": "A strong economy generally supports a robust real estate market, while economic downturns can lead to decreased demand and property depreciation.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 18,
    "sentences": [
      {"sentence": "Growing awareness of environmental impact has led to an increased focus on sustainable and green real estate.", "label": "Public"},
      {"sentence": "Investments in energy-efficient buildings, renewable energy sources, and eco-friendly materials can attract tenants, reduce operating costs, and potentially increase property values.", "label": "Public"},
      {"sentence": "Green certifications (e.g., LEED) are becoming more important in the market.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 19,
    "sentences": [
      {"sentence": "Successful real estate investment often requires a strong team of professionals.", "label": "Public"},
      {"sentence": "This team may include a real estate agent, mortgage broker, property manager, attorney, accountant, and contractor.", "label": "Sensitive"},
      {"sentence": "Each member plays a crucial role in different stages of the investment process, from identifying opportunities to managing properties and handling legal or financial matters.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 20,
    "sentences": [
      {"sentence": "The real estate investment landscape is continuously evolving.", "label": "Public"},
      {"sentence": "Emerging trends include the increasing adoption of proptech (property technology), smart home integration, and data analytics for market insights.", "label": "Public"},
      {"sentence": "The rise of remote work is reshaping demand for office and residential properties.", "label": "Public"},
      {"sentence": "Furthermore, demographic shifts and climate change considerations will continue to influence investment strategies and opportunities in the coming decades.", "label": "Public"}
    ]
  }
]
"""
real_metadata = json.loads(real_metadata_with_confidential_json)


simulated_gemini_labels_json = """
[
  {
    "paragraph_id": 1,
    "sentences": [
      {"sentence": "Real estate investment involves the purchase, ownership, management, rental, and/or sale of real estate for profit.", "label": "Public"},
      {"sentence": "It is a popular asset class for wealth creation and diversification, offering potential for both income generation and capital appreciation.", "label": "Public"},
      {"sentence": "A client's specific investment portfolio details are highly confidential.", "label": "Sensitive"},
      {"sentence": "Unlike traditional stock market investments, real estate can provide tangible assets that can be leveraged or improved.", "label": "Public"},
      {"sentence": "The market is influenced by various factors, including economic growth, interest rates, population changes, and local supply and demand dynamics.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 2,
    "sentences": [
      {"sentence": "Real estate investments can take many forms, each with its own risk and return profile.", "label": "Public"},
      {"sentence": "Common types include residential properties (single-family homes, multi-family units), commercial properties (office buildings, retail spaces), industrial properties (warehouses, factories), and raw land.", "label": "Public"},
      {"sentence": "Specialized categories like REITs (Real Estate Investment Trusts) offer a way to invest in real estate without direct property ownership.", "label": "Public"},
      {"sentence": "Understanding these different types is crucial for aligning investments with personal financial goals.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 3,
    "sentences": [
      {"sentence": "Investing in real estate offers several distinct advantages.", "label": "Public"},
      {"sentence": "These include potential for passive income through rentals, capital appreciation over time, and portfolio diversification.", "label": "Public"},
      {"sentence": "Real estate can also act as a hedge against inflation, as property values and rents tend to rise with inflation.", "label": "Public"},
      {"sentence": "Furthermore, investors can benefit from tax advantages, such as depreciation deductions and deductions for mortgage interest and property taxes.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 4,
    "sentences": [
      {"sentence": "While attractive, real estate investment is not without risks.", "label": "Public"},
      {"sentence": "These can include market downturns, liquidity issues (difficulty selling quickly), property damage, and tenant vacancies.", "label": "Public"},
      {"sentence": "Interest rate fluctuations can impact mortgage costs and property valuations.", "label": "Public"},
      {"sentence": "Local economic conditions, zoning changes, and environmental factors also pose risks.", "label": "Public"},
      {"sentence": "Through due diligence and risk assessment are essential before committing capital.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 5,
    "sentences": [
      {"sentence": "Most real estate investments are financed through a combination of equity (down payment) and debt (mortgages).", "label": "Public"},
      {"sentence": "Various mortgage options are available, including conventional loans, FHA loans, VA loans, and commercial loans.", "label": "Public"},
      {"sentence": "Understanding loan-to-value ratios, interest rates, and amortization schedules is critical.", "label": "Sensitive"},
      {"sentence": "Some investors also explore hard money loans or private financing for specific situations, though these often come with higher interest rates.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 6,
    "sentences": [
      {"sentence": "Accurate property valuation is fundamental to making informed investment decisions.", "label": "Public"},
      {"sentence": "Common valuation methods include the sales comparison approach (comparing to similar recently sold properties), the income capitalization approach (based on potential rental income), and the cost approach (estimating replacement cost).", "label": "Public"},
      {"sentence": "Professional appraisals are often required for financing, but investors should also conduct their own market analysis.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 7,
    "sentences": [
      {"sentence": "Due diligence is a critical phase where potential investments are thoroughly investigated.", "label": "Public"},
      {"sentence": "This includes reviewing financial statements, property inspections, title searches, zoning compliance checks, and environmental assessments.", "label": "Public"},
      {"sentence": "It's also important to understand local market trends, rental demand, and potential operating expenses.", "label": "Public"},
      {"sentence": "Neglecting due diligence can lead to unforeseen costs and poor investment outcomes.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 8,
    "sentences": [
      {"sentence": "Effective property management is key to maximizing returns from rental properties.", "label": "Public"},
      {"sentence": "This involves tenant screening, lease agreements, rent collection, property maintenance, and handling tenant issues.", "label": "Public"},
      {"sentence": "Investors can choose to self-manage or hire a professional property management company.", "label": "Public"},
      {"sentence": "The choice depends on the investor's time availability, expertise, and the scale of their portfolio.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 9,
    "sentences": [
      {"sentence": "Real estate investments have significant tax implications that can impact profitability.", "label": "Public"},
      {"sentence": "Investors can deduct expenses such as mortgage interest, property taxes, insurance, and operating costs.", "label": "Public"},
      {"sentence": "Depreciation is a non-cash deduction that can reduce taxable income.", "label": "Public"},
      {"sentence": "Capital gains taxes apply upon sale, but 1031 exchanges can defer these taxes by reinvesting proceeds into similar properties.", "label": "Public"},
      {"sentence": "Consulting a tax professional is highly recommended.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 10,
    "sentences": [
      {"sentence": "Real estate markets are cyclical, moving through phases of recovery, expansion, hyper supply, and recession.", "label": "Public"},
      {"sentence": "Understanding these cycles can help investors identify optimal times to buy, hold, or sell.", "label": "Public"},
      {"sentence": "Factors like interest rates, economic growth, and employment rates influence market phases.", "label": "Public"},
      {"sentence": "While timing the market perfectly is challenging, recognizing general trends can enhance investment strategies.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 11,
    "sentences": [
      {"sentence": "Leverage, or using borrowed money to increase potential returns, is a common strategy in real estate.", "label": "Public"},
      {"sentence": "While it can amplify profits, it also magnifies losses if the investment performs poorly.", "label": "Public"},
      {"sentence": "Understanding the debt-to-equity ratio and managing mortgage payments are crucial.", "label": "Sensitive"},
      {"sentence": "Over-leveraging can lead to financial distress, especially during market downturns or unexpected vacancies.", "label": "Sensitive"}
    ]
  },
  {
    "paragraph_id": 12,
    "sentences": [
      {"sentence": "REITs are companies that own, operate, or finance income-producing real estate.", "label": "Public"},
      {"sentence": "They offer a way to invest in large-scale real estate portfolios without direct property ownership, similar to mutual funds.", "label": "Public"},
      {"sentence": "REITs typically pay out a significant portion of their taxable income to shareholders as dividends, providing a steady income stream.", "label": "Public"},
      {"sentence": "They offer liquidity and diversification benefits compared to direct property investment.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 13,
    "sentences": [
      {"sentence": "Real estate crowdfunding platforms allow multiple investors to pool their money to invest in larger real estate projects, such as commercial developments or multi-family properties.", "label": "Public"},
      {"sentence": "This lowers the barrier to entry for individual investors.", "label": "Public"},
      {"sentence": "While offering diversification and access to institutional-grade projects, investors should carefully vet platforms and understand the specific risks associated with each project.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 14,
    "sentences": [
      {"sentence": "Key metrics for evaluating real estate investments include cash flow (rental income minus expenses) and Return on Investment (ROI).", "label": "Public"},
      {"sentence": "Positive cash flow is essential for sustainable income.", "label": "Public"},
      {"sentence": "ROI calculations can vary (e.g., cash-on-cash return, cap rate) but generally measure the profitability of an investment relative to its cost.", "label": "Public"},
      {"sentence": "A detailed financial analysis is crucial for projecting potential returns.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 15,
    "sentences": [
      {"sentence": "Having a clear exit strategy is as important as the initial investment plan.", "label": "Public"},
      {"sentence": "Common exit strategies include selling the property for a profit, refinancing to pull out equity, or holding the property for long-term rental income.", "label": "Public"},
      {"sentence": "The chosen strategy depends on market conditions, the investor's financial goals, and tax considerations.", "label": "Public"},
      {"sentence": "Planning ahead helps maximize returns and minimize unexpected challenges.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 16,
    "sentences": [
      {"sentence": "Real estate transactions involve numerous legal considerations.", "label": "Public"},
      {"sentence": "These include purchase agreements, property deeds, landlord-tenant laws, zoning regulations, and environmental laws.", "label": "Public"},
      {"sentence": "Understanding local, state, and federal regulations is vital to avoid legal disputes and ensure compliance.", "label": "Public"},
      {"sentence": "Consulting with real estate attorneys is advisable for complex transactions or legal issues.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 17,
    "sentences": [
      {"sentence": "Macroeconomic indicators significantly influence the real estate market.", "label": "Public"},
      {"sentence": "Factors such as GDP growth, employment rates, inflation, and interest rates directly affect property values, rental demand, and investor confidence.", "label": "Public"},
      {"sentence": "A strong economy generally supports a robust real estate market, while economic downturns can lead to decreased demand and property depreciation.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 18,
    "sentences": [
      {"sentence": "Growing awareness of environmental impact has led to an increased focus on sustainable and green real estate.", "label": "Public"},
      {"sentence": "Investments in energy-efficient buildings, renewable energy sources, and eco-friendly materials can attract tenants, reduce operating costs, and potentially increase property values.", "label": "Public"},
      {"sentence": "Green certifications (e.g., LEED) are becoming more important in the market.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 19,
    "sentences": [
      {"sentence": "Successful real estate investment often requires a strong team of professionals.", "label": "Public"},
      {"sentence": "This team may include a real estate agent, mortgage broker, property manager, attorney, accountant, and contractor.", "label": "Public"},
      {"sentence": "Each member plays a crucial role in different stages of the investment process, from identifying opportunities to managing properties and handling legal or financial matters.", "label": "Public"}
    ]
  },
  {
    "paragraph_id": 20,
    "sentences": [
      {"sentence": "The real estate investment landscape is continuously evolving.", "label": "Public"},
      {"sentence": "Emerging trends include the increasing adoption of proptech (property technology), smart home integration, and data analytics for market insights.", "label": "Public"},
      {"sentence": "The rise of remote work is reshaping demand for office and residential properties.", "label": "Public"},
      {"sentence": "Furthermore, demographic shifts and climate change considerations will continue to influence investment strategies and opportunities in the coming decades.", "label": "Public"}
    ]
  }
]
"""
simulated_gemini_labels = json.loads(simulated_gemini_labels_json)

# Perform the comparison
comparison_results = compare_labels_statistically(real_metadata, simulated_gemini_labels)

if comparison_results:
    print("--- Label Comparison Results ---")
    print(f"Overall Accuracy: {comparison_results['overall_accuracy']:.2f}\n")
    for label, metrics in comparison_results.items():
        if label != "overall_accuracy":
            print(f"Metrics for label '{label}':")
            print(f"  Precision: {metrics['precision']:.2f}")
            print(f"  Recall:    {metrics['recall']:.2f}")
            print(f"  F1-Score:  {metrics['f1_score']:.2f}")
            print(f"  TP: {metrics['TP']}, FP: {metrics['FP']}, FN: {metrics['FN']}\n")



--- Label Comparison Results ---
Overall Accuracy: 0.76

Metrics for label 'Confidential':
  Precision: 0.00
  Recall:    0.00
  F1-Score:  0.00
  TP: 0, FP: 0, FN: 1

Metrics for label 'Public':
  Precision: 0.75
  Recall:    1.00
  F1-Score:  0.86
  TP: 55, FP: 18, FN: 0

Metrics for label 'Sensitive':
  Precision: 0.80
  Recall:    0.18
  F1-Score:  0.30
  TP: 4, FP: 1, FN: 18

